In [2]:
# testing python speed
import numpy as np
from numba import njit
import timeit


@njit
def kro_loop(sw, kro0, sor, swc, no):
    res = np.zeros_like(sw)
    for i in range(sw.shape[0]):
        for j in range(sw.shape[1]):
            if swc <= sw[i, j] <= 1 - sor:
                res[i, j] = kro0 * ((1 - sw[i, j] - sor) / (1 - sor - swc)) ** no
            elif 0.0 < sw[i, j] < swc:
                res[i, j] = 1 + (kro0 - 1) / swc * sw[i, j]
            elif sw[i, j] > 1 - sor:
                res[i, j] = 0.0
            elif sw[i, j] <= 0.0:
                res[i, j] = 1.0
    return res

def kro(sw, kro0, sor, swc, no):
    return (np.where((swc <= sw) & (sw <= 1 - sor), kro0 * ((1 - sw - sor) / (1 - sor - swc)) ** no, 0.0)+
        np.where((0.0 < sw) & (sw < swc), 1 + (kro0 - 1) / swc * sw, 0.0)+
        np.where(sw > 1 - sor, 0.0, 0.0)+
        np.where(sw <= 0.0, 1.0, 0.0)).astype(float)

def mykro(sw, kro0, sor, swc, no):
    res = ((swc<=sw) & (sw<=1-sor))*kro0*((1-sw-sor)/(1-sor-swc))**no \
    +((0.0<sw) & (sw<swc))*(1+(kro0-1)/swc*sw) \
    +(sw>1-sor)*0.0 \
    +(sw<=0.0)*1.0
    return res

def kro_arg(sw, kro0, sor, swc, no):
    res = np.zeros_like(sw)
    
    indices = np.argwhere((swc <= sw) & (sw <= 1 - sor))
    res[indices[:, 0], indices[:, 1]] = kro0 * ((1 - sw[indices[:, 0], indices[:, 1]] - sor) / (1 - sor - swc)) ** no
    
    indices = np.argwhere((0.0 < sw) & (sw < swc))
    res[indices[:, 0], indices[:, 1]] = 1 + (kro0 - 1) / swc * sw[indices[:, 0], indices[:, 1]]
    
    indices = np.argwhere(sw > 1 - sor)
    res[indices[:, 0], indices[:, 1]] = 0.0
    
    indices = np.argwhere(sw <= 0.0)
    res[indices[:, 0], indices[:, 1]] = 1.0
    return res

def kro_logic(sw, kro0, sor, swc, no):
    res = np.zeros_like(sw)
    cond1 = np.logical_and(swc <= sw, sw <= 1 - sor)
    res[(swc <= sw) & (sw <= 1 - sor)] = kro0 * ((1 - sw[cond1] - sor) / (1 - sor - swc)) ** no
    cond2 = np.logical_and(0.0 < sw, sw < swc)
    res[cond2] = 1 + (kro0 - 1) / swc * sw[cond2]
    res[sw > 1 - sor] = 0.0
    res[sw <= 0.0] = 1.0
    return res

sw = np.linspace(0.0, 1.0, 100000).reshape(100,1000)
kro_time = lambda: kro_loop(sw, 0.9, 0.05, 0.1, 2.0)
kro_time2 = lambda: kro(sw, 0.9, 0.05, 0.1, 2.0)
kro_time3 = lambda: mykro(sw, 0.9, 0.05, 0.1, 2.0)
kro_time4 = lambda: kro_arg(sw, 0.9, 0.05, 0.1, 2.0)
kro_time5 = lambda: kro_logic(sw, 0.9, 0.05, 0.1, 2.0)
num = 10000
elapsed_time_loop = timeit.timeit(kro_time, number=num)
elapsed_time_loop2 = timeit.timeit(kro_time2, number=num)
elapsed_time_loop3 = timeit.timeit(kro_time3, number=num)
elapsed_time_loop4 = timeit.timeit(kro_time4, number=num)
elapsed_time_loop5 = timeit.timeit(kro_time5, number=num)


# Print the elapsed time
print(f"Elapsed time: {elapsed_time_loop} seconds")
print(f"where Elapsed time: {elapsed_time_loop2} seconds")
print(f"my Elapsed time: {elapsed_time_loop3} seconds")
print(f"argwhere Elapsed time: {elapsed_time_loop4} seconds")
print(f"logic Elapsed time: {elapsed_time_loop5} seconds")


Elapsed time: 16.04552859999967 seconds
where Elapsed time: 5.862902799999574 seconds
my Elapsed time: 6.189596399999573 seconds
argwhere Elapsed time: 33.608150099999875 seconds
logic Elapsed time: 3.4545826000003217 seconds


In [21]:
kro(np.array([0.3, 0.2, 0.01]), 0.8, 0.2, 0.1, 2.0)

array([0.40816327, 0.5877551 , 0.98      ])

In [22]:
import timeit

# Define the setup code (imports and function definition)
setup_code = """
import numpy as np
from numba import njit

def kro_orig(sw, kro0, sor, swc, no):
    condition1 = (swc <= sw) & (sw <= 1 - sor)
    condition2 = (0.0 < sw) & (sw < swc)
    condition3 = sw > 1 - sor
    condition4 = sw <= 0.0

    term1 = np.where(condition1, kro0 * ((1 - sw - sor) / (1 - sor - swc)) ** no, 0.0)
    term2 = np.where(condition2, 1 + (kro0 - 1) / swc * sw, 0.0)
    term3 = np.where(condition3, 0.0, 0.0)
    term4 = np.where(condition4, 1.0, 0.0)

    res = term1 + term2 + term3 + term4
    return res

def kro(sw, kro0, sor, swc, no):
    return (np.where((swc <= sw) & (sw <= 1 - sor), kro0 * ((1 - sw - sor) / (1 - sor - swc)) ** no, 0.0)+
        np.where((0.0 < sw) & (sw < swc), 1 + (kro0 - 1) / swc * sw, 0.0)+
        np.where(sw > 1 - sor, 0.0, 0.0)+
        np.where(sw <= 0.0, 1.0, 0.0)).astype(float)

def mykro(sw, kro0, sor, swc, no):
    res = ((swc<=sw) & (sw<=1-sor))*kro0*((1-sw-sor)/(1-sor-swc))**no \
    +((0.0<sw) & (sw<swc))*(1+(kro0-1)/swc*sw) \
    +(sw>1-sor)*0.0 \
    +(sw<=0.0)*1.0
    return res
"""

# Define the code snippet to measure the runtime
code_to_test = """
# Call the function with sample inputs
sw = np.linspace(0.0, 1.0, 100000).reshape(100,1000)
kro(sw, 0.9, 0.05, 0.1, 2.0)
"""

mycode_to_test = """
# Call the function with sample inputs
sw = np.linspace(0.0, 1.0, 100000).reshape(100,1000)
mykro(sw, 0.9, 0.05, 0.1, 2.0)
"""

origcode_to_test = """
# Call the function with sample inputs
sw = np.linspace(0.0, 1.0, 100000).reshape(100,1000)
kro_orig(sw, 0.9, 0.05, 0.1, 2.0)
"""

# Measure the runtime
execution_time = timeit.timeit(code_to_test, setup=setup_code, number=2000)
myexecution_time = timeit.timeit(mycode_to_test, setup=setup_code, number=2000)
origexecution_time = timeit.timeit(origcode_to_test, setup=setup_code, number=2000)

print(f"original Execution time: {origexecution_time} seconds")
print(f"Execution time: {execution_time} seconds")
print(f"My Execution time: {myexecution_time} seconds")


original Execution time: 7.708333700000367 seconds
Execution time: 6.949330499999633 seconds
My Execution time: 6.562617599996884 seconds


In [10]:
np.linspace(0.0, 1.0, 100000).reshape(100,1000)

array([[0.00000000e+00, 1.00001000e-05, 2.00002000e-05, ...,
        9.97009970e-03, 9.98009980e-03, 9.99009990e-03],
       [1.00001000e-02, 1.00101001e-02, 1.00201002e-02, ...,
        1.99701997e-02, 1.99801998e-02, 1.99901999e-02],
       [2.00002000e-02, 2.00102001e-02, 2.00202002e-02, ...,
        2.99702997e-02, 2.99802998e-02, 2.99902999e-02],
       ...,
       [9.70009700e-01, 9.70019700e-01, 9.70029700e-01, ...,
        9.79979800e-01, 9.79989800e-01, 9.79999800e-01],
       [9.80009800e-01, 9.80019800e-01, 9.80029800e-01, ...,
        9.89979900e-01, 9.89989900e-01, 9.89999900e-01],
       [9.90009900e-01, 9.90019900e-01, 9.90029900e-01, ...,
        9.99980000e-01, 9.99990000e-01, 1.00000000e+00]])

Another way to do these alculations is table reading. Let's try it now.

In [13]:
from scipy.interpolate import interp1d
sw_interp = np.linspace(0.0, 1.0, 100)
kro_values = kro(sw_interp, 0.9, 0.05, 0.1, 2.4)
kro_interp = interp1d(sw_interp, kro_values, kind='linear', bounds_error=False, fill_value=(kro_values[0], kro_values[-1]))
sw = np.linspace(0.0, 1.0, 100000).reshape(100,1000)
kro_i = lambda: kro_interp(sw)
elapsed_time_int = timeit.timeit(kro_i, number=num)
print(f"Elapsed time: {elapsed_time_int} seconds")

C:\Users\aaeft\AppData\Local\Temp\ipykernel_16572\3619572297.py:23: RuntimeWarning: invalid value encountered in power
  return (np.where((swc <= sw) & (sw <= 1 - sor), kro0 * ((1 - sw - sor) / (1 - sor - swc)) ** no, 0.0)+


Elapsed time: 4.8909181999988505 seconds


In [1]:
import numpy as np

In [3]:
np.max([2,3,4])

4

In [5]:
# assign the values of a numpy array to a variable
# and plot the values of the variable
# using the matplotlib library
# import the numpy library
import numpy as np
# import the matplotlib library
import matplotlib.pyplot as plt
# create a numpy array
x = np.array([1, 2, 3, 4])
# assign the values of the numpy array to a variable
y = x
# plot the values of the variable
plt.plot(x, y)
# show the plot
plt.show()



In [10]:
x = np.array([1, 2, 3, 4])
y = 2*np.array([1, 2, 3, 4])

In [11]:
x[:] = y

In [12]:
x

array([2, 4, 6, 8])

In [13]:
y

array([2, 4, 6, 8])

In [14]:
y[0] = 0
x

array([2, 4, 6, 8])

In [15]:
y

array([0, 4, 6, 8])